In [2]:
import pandas as pd
from numpy import genfromtxt
from PIL import Image
import torch
import numpy as np
from copy import deepcopy as copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [3]:
train = np.load('training_data.npy', allow_pickle=True)

In [4]:
print(len(train))

37950


In [5]:
indices = np.random.choice(37950, 3795)
test = train[indices]

In [6]:
train_X = torch.Tensor([i[1] for i in train]).view(-1, 28, 28)
train_Y = torch.Tensor([i[0] for i in train])
test_X = torch.Tensor([i[1] for i in test]).view(-1, 28, 28)
test_Y = torch.Tensor([i[0] for i in test])

In [7]:
class Net(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 28, 3)
            self.pool1 = nn.MaxPool2d((2, 2))
            self.fc1 = nn.Linear(4732, 10)
        
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool1(x)
            x = x.flatten(start_dim=1)
            x = self.fc1(x)
            return F.softmax(x, dim=1)

In [43]:
device = torch.device('cuda:0')
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

BATCH_SIZE = 100
EPOCHS = 3

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 28, 28).to(device)
        batch_Y = train_Y[i:i+BATCH_SIZE].to(device)
        net.zero_grad()
        optimizer.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_Y)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch}. Loss: {loss}")

 16%|█▌        | 59/380 [00:00<00:00, 589.87it/s]

Epoch: 0. Loss: 0.0014420648803934455


 16%|█▌        | 60/380 [00:00<00:00, 593.93it/s]

Epoch: 1. Loss: 0.0004987588617950678


100%|██████████| 380/380 [00:00<00:00, 603.04it/s]

Epoch: 2. Loss: 0.0003792305360548198


In [47]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_Y[i]).to(device)
        net_out = net(test_X[i].view(-1, 1, 28, 28).to(device))[0]
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1

print("Accuracy: ", round(correct/total, 3))

100%|██████████| 3795/3795 [00:01<00:00, 1943.72it/s]

Accuracy:  0.982


In [48]:
torch.save(net.state_dict(), 'model1')

In [10]:
device = torch.device('cuda:0')
net = Net().to(device)
net.load_state_dict(torch.load('model1'))
net.eval()

Net(
  (conv1): Conv2d(1, 28, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4732, out_features=10, bias=True)
)

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_Y[i]).to(device)
        net_out = net(test_X[i].view(-1, 1, 28, 28).to(device))[0]
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1

print("Accuracy: ", round(correct/total, 3))

100%|██████████| 3795/3795 [00:03<00:00, 1212.19it/s]

Accuracy:  0.977
